In [6]:
from google.cloud import vision
from google.oauth2 import service_account
from google.cloud import vision_v1
from google.protobuf import json_format
import os
import re

key_path = '/Users/pradeepsain/Downloads/KEY_FILE.json'
credentials = service_account.Credentials.from_service_account_file(key_path)
client = vision.ImageAnnotatorClient(credentials=credentials)

directory = '/Users/pradeepsain/Downloads/dash_img/'

for i in range(1, 50):
    filename = f'readingImg_{i}.jpg'
    img_path = os.path.join(directory, filename)
    
    with open(img_path, 'rb') as image_file:
        content = image_file.read()
    image = vision_v1.Image(content=content)
    image_context = vision_v1.ImageContext(language_hints=["en"])

    response = client.text_detection(image=image, image_context=image_context)
    extracted_text = response.text_annotations

    cleaned_text = ""
    for text in extracted_text:
        text_description = text.description
        if isinstance(text_description, str):
            cleaned_text += re.sub(r'[^\w\s./:]', '', text_description) + " "

    text_in_one_line = " ".join(cleaned_text.split())

    with open('/Users/pradeepsain/Downloads/TextFromImages.txt', 'a') as f:
        f.write(text_in_one_line + "\n")


In [39]:
import json
import csv

json_file_path = '/Users/pradeepsain/Downloads/TextFromImages.json'
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

dataset = []

annotations = data.get("annotations", [])
for entry in annotations:
    if entry is not None:
        text = entry[0]
        entities = entry[1]['entities']

        dataset.append((text, entities))

output_csv_path = '/Users/pradeepsain/Downloads/TextFromImages.csv'

header = ['Text', 'Start', 'End', 'Label']

with open(output_csv_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)

    writer.writerow(header)

    for entry in dataset:
        text = entry[0]
        entities = entry[1]

        for entity in entities:
            start, end, label = entity
            writer.writerow([text, start, end, label])


In [18]:
# import spacy
# from spacy.training.example import Example

# # nlp = spacy.blank("en")
# # nlp = spacy.load("en_core_web_sm")
# # labels = ["BG READING", "UNITS", "DATE", "TIME", "TIMESTAMP", "BG TEXT"]
# # ner = nlp.add_pipe("ner", config={"labels": labels})
# # nlp.add_pipe("ner", config={"labels": ["BG READING", "UNITS","DATE","TIME","TIMESTAMP","BG TEXT"]})

# nlp = spacy.load("en_core_web_sm")
# ner = nlp.get_pipe("ner")
# custom_labels = ["BG READING", "UNITS", "DATE", "TIME", "TIMESTAMP", "BG TEXT"]
# for label in custom_labels:
#     ner.add_label(label)

# TRAIN_DATA = output_csv_path

# optimizer = nlp.begin_training()
# for epoch in range(10):
#     random.shuffle(TRAIN_DATA)
#     for texts, annotations in TRAIN_DATA:
#         example = Example.from_dict(nlp.make_doc(texts), annotations)
#         nlp.update([example], drop=0.5, losses={})

# nlp.to_disk("/Users/pradeepsain/Downloads/trained_model")

In [44]:
# import spacy
# from spacy.training.example import Example

# nlp1 = spacy.blank("en")

# labels = ["BG READING", "UNITS", "DATE", "TIME", "TIMESTAMP", "BG TEXT"]
# nlp1.add_pipe("ner", config={"labels": labels})

# TRAIN_DATA = pd.read_csv(output_csv_path)
# # TRAIN_DATA = json.loads(json_file_path)

# optimizer = nlp1.begin_training()
# for epoch in range(10):
#     random.shuffle(TRAIN_DATA)
#     for texts, annotations in TRAIN_DATA:
#         example = Example.from_dict(nlp1.make_doc(texts), annotations)
#         nlp1.update([example], drop=0.5, losses={})

# nlp1.to_disk("/Users/pradeepsain/Downloads/trained_model")

In [45]:
# import spacy
# from spacy.training.example import Example
# import json
# import random

# nlp1 = spacy.blank("en")

# labels = ["BG READING", "UNITS", "DATE", "TIME", "TIMESTAMP", "BG TEXT"]

# nlp1.add_pipe("ner", config={"labels": labels})

# with open(json_file_path, "r") as json_file:
#     TRAIN_DATA = json.load(json_file)

# optimizer = nlp1.begin_training()

# for epoch in range(10):
#     random.shuffle(TRAIN_DATA)
#     losses = {}
#     for texts, annotations in TRAIN_DATA:
#         example = Example.from_dict(nlp1.make_doc(texts), annotations)
        
#         nlp1.update([example], drop=0.5, losses=losses)
    
#     print("Epoch:", epoch, "Losses:", losses)

# nlp1.to_disk("/Users/pradeepsain/Downloads/trained_model")


In [58]:
# print("Labels specified in your code:", labels)
# # print("Labels specified in spaCy configuration:", nlp1.get_pipe("ner").labels)

In [57]:
# nlp3 = spacy.blank("en")

# labels = ["BG READING", "UNITS", "DATE", "TIME", "TIMESTAMP", "BG TEXT"]
# # ner3 = nlp3.add_pipe("ner", config={"labels": labels})

In [41]:
# from spacy.pipeline.textcat import single_label_cnn_config
# nlp=spacy.load("en_core_web_sm")
# config = Config().from_str(single_label_cnn_config)
# if "textcat" not in nlp.pipe_names:
#      nlp.add_pipe('textcat', config=config, last=True)

# nlp.pipe_names

In [42]:
# nlp = spacy.load("en_core_web_sm")
# if "textcat" not in nlp.pipe_names:
#      nlp.add_pipe('textcat', config=single_label_cnn_config, last=True)
# textcat = nlp.get_pipe('textcat')
# textcat.add_label("pos")
# textcat.add_label("neg")

In [47]:
import pandas as pd
import spacy
from spacy.training.example import Example

df = pd.read_csv(output_csv_path)

nlp = spacy.blank('en') 

def create_training_data(df):
    training_data = []
    for _, row in df.iterrows():
        text = row['Text']
        start = row['Start']
        end = row['End']
        label = row['Label']

        entities = [(start, end, label)]

        example = Example.from_dict(nlp.make_doc(text), {"entities": entities})
        training_data.append(example)
    return training_data

training_data = create_training_data(df)

In [48]:
ner = nlp.add_pipe('ner')

unique_labels = df['Label'].unique()
for label in unique_labels:
    ner.add_label(label)

nlp.begin_training()

for example in training_data:
    nlp.update([example], drop=0.5)

nlp.to_disk('/Users/pradeepsain/Downloads/ner_model')

In [49]:
# import pandas as pd
# import spacy
# from spacy.training.example import Example

# # Load CSV data
# df = pd.read_csv(output_csv_path)

# nlp = spacy.blank('en')

# def create_training_data(df):
#     training_data = []
#     for _, row in df.iterrows():
#         text = row['text']
#         annotations = row['annotation']  #.split('|')
#         entities = []
#         for annotation in annotations:
#             entity, label = annotation   #.split('|')
#             start = text.find(entity)
#             end = start + len(entity)
#             entities.append((start, end, label))
#         example = Example.from_dict(nlp.make_doc(text), {"entities": entities})
#         training_data.append(example)
#     return training_data

# training_data = create_training_data(df)

In [100]:
import spacy

nlp_trained = spacy.load("/Users/pradeepsain/Downloads/ner_model")

with open('/Users/pradeepsain/Downloads/dash_img/readingImg_6.jpg', 'rb') as image_file:
    content = image_file.read()

image = vision_v1.Image(content=content)
image_context = vision_v1.ImageContext(language_hints=["en"])

response = client.text_detection(image=image, image_context=image_context)
extracted_text = response.text_annotations

cleaned_text = ""
for text in extracted_text:
    text_description = text.description
    if isinstance(text_description, str):
        cleaned_text += re.sub(r'[^\w\s./:]', '', text_description) + " "

one_line_text = " ".join(cleaned_text.split())

print(one_line_text)

doc = nlp_trained(one_line_text)


readings = [ent.text for ent in doc.ents if ent.label_ == "BG READING"]
units = [ent.text for ent in doc.ents if ent.label_ == "UNITS"]
date = [ent.text for ent in doc.ents if ent.label_ == "DATE"]
time = [ent.text for ent in doc.ents if ent.label_ == "TIME"]
timestamp = [ent.text for ent in doc.ents if ent.label_ == "TIMESTAMP"]
bgText = [ent.text for ent in doc.ents if ent.label_ == "BG TEXT"]

print("Detected Readings:", readings)
print("Detected units:", units)
print("Detected date:", date)
print("Detected time:", time)
print("Detected timestamp:", timestamp)
print("Detected bgText:", bgText)

No Pod Dashboard Sep 26 Basal LAST BOLUS Date/Time change occurred X No Active Pod SET UP NEW POD HO 84 12:14 PM Bolus C Pod Info LAST BG 372 mg/dL Today Right Now 0 No Pod Dashboard Sep 26 Basal LAST BOLUS Date / Time change occurred X No Active Pod SET UP NEW POD HO 84 12:14 PM Bolus C Pod Info LAST BG 372 mg / dL Today Right Now 0
Detected Readings: []
Detected units: []
Detected date: []
Detected time: []
Detected timestamp: []
Detected bgText: []


In [101]:
print("Detected Readings:", [ent.text for ent in doc.ents])

Detected Readings: []
